In [2]:
import numpy as np
from random import gauss
import pandas as pd

import scipy.optimize as sp
import math

import pandas as pd

In [ ]:
# A21():
#     []
# #different projector products are 0
# #they are normalised
# #projector product is itself

# A1 ={[1,0,0,0],[0,1,0,0]} #2 dimensions
# A2 = {[a,b],[e,f],[]}

In [2]:
def f(x):
    return -1*(( (math.sin(x[0])**2) * (math.cos(x[0]))**2 * (math.cos(x[1]))**2 * (math.sin(x[1])**2) * 
             ((math.sin(x[1])**2) - (math.cos(x[1]))**2)**2)
            /(((math.sin(x[1])**3)*math.sin(x[0]))**2 + ((math.cos(x[1])**3)*(math.cos(x[0])))**2))

In [3]:
bnds = ((-1*math.pi, math.pi), (-1*math.pi, math.pi))
x0 = [1,2]

In [13]:
result = sp.minimize(f, x0, tol=1e-15, bounds = bnds) 

In [14]:
result

      fun: -0.09016994374947422
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([1.38777879e-09, 2.77555758e-09])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 45
      nit: 11
     njev: 15
   status: 0
  success: True
        x: array([-1.26437613,  2.70690031])

In [5]:
print(-1*result.fun)

0.09016994374947422


In [5]:
def innerproduct(x,y):

    temp = 0

    for i in range(len(x)):
        temp+= x[i]*y[i]

    return np.array(temp)
  
def tensorproduct(x,y):

    temp = []

    for i in range(len(x)):
        for j in range(len(x)):
            temp.append(x[i]*y[j])

    return np.array(temp)
  
def outerproduct(x,y):

    return np.outer(x,y)
  
def trace(mat):

    return np.trace(mat)

def make_rand_vector(dims):
    vec = [gauss(0, 1) for i in range(dims**2)]
    mag = sum(x**2 for x in vec) ** .5
    return np.array([x/mag for x in vec]).reshape((2,2))

def basis(e,d):
    temp = np.zeros(d).astype(int)
    temp[e] = 1
    return np.array(temp)


def kronecker_product(x,y):

    return np.kron(x,y)

In [7]:
# P(A2 < B1) = 0
# P(B1 < A1) = 0
# P(A1 < B2) = 0

# P(A2 < B2) > 0

# H is a dxd upper triangular matrix because U1,V1 projectors are identity hence 

# def Rho(d):
#      return np.random.rand(d*d).reshape(d,d)

# x = [rho,A2,B2]

#A1=I, B1=I

# def A11():
    
def A1(d):
    temp = []
    for i in range(d):
        temp.append(outerproduct(basis(i,d),basis(i,d)))
    return np.array(temp)


def B1(d):
    temp = []
    for i in range(d):
        temp.append(outerproduct(basis(i,d),basis(i,d)))
    return np.array(temp)

def Inequalities(d):
    
    ans = []

    for i in range(1,d):
        for j in range(i+1,d+1):
            ans.append([i-1,j-1])
    
    return np.array(ans)

def Projector(Matrix,state):
    temp = outerproduct(state,state)
    temp2 = np.matmul(Matrix, temp)
    
    return np.matmul(temp2,Matrix.transpose())

def Probability(A,m,B,n,rho,d):
    e1 = basis(m,d)
    e2 = basis(n,d)
    
    A = A.reshape(d,d)
    B = B.reshape(d,d)
    
    temp = kronecker_product(Projector(A,e1), Projector(B,e2))
    
    temp2 = temp.dot(outerproduct(rho,rho).transpose())
    
    return trace(temp2)

#P(X<Y) = P(X=0,Y=1) for d=2
# for d=3 (P(X=0,Y=1),P(X=0,Y=2)), P(X=1,Y=2)
def Probability_Inequality(A,B,rho,d):
    s = 0
    
    for (i,j) in Inequalities(d):
        s += Probability(A,i,B,j,rho,d)
    
    return s

def const1(x,d):
    return Probability_Inequality(B1(d),A1(d),x[:d**2],d) #P(b1<a1) = 0

def const2(x,d):
    return Probability_Inequality(A1(d),x[2*d**2:],x[:d**2],d) #P(A1<B2) = 0

def const3(x,d):
    return Probability_Inequality(x[d**2:2*d**2],B1(d),x[:d**2],d) #P(A2<B1) = 0

def const5(x,d):
    return sum([a*b for a,b in zip(x[:d**2],x[:d**2])]) - 1 #h is normalised

def const6(x,d):
    return sum([a*b for a,b in zip(x[d**2:d**2+d],x[d**2:d**2+d])]) - 1 # #A2 is SU(d)

def const7(x,d):
    return sum([a*b for a,b in zip(x[2*d**2:2*d**2 +d],x[2*d**2:2*d**2 +d])]) - 1 #B2 is SU(d)

def objective(x,d):
    return -1*(Probability_Inequality(x[d**2:2*d**2],x[2*d**2:],x[:d**2],d)) #P(A2<B2)


In [8]:
def main(d):
    
    x0 = np.random.rand(3*d*d)
    
    eq_constraints = [{'type': 'eq',
                    'fun' : const1,
                      "args" : (d,)},
                      
                 {'type': 'eq',
                    'fun' : const2,
                      "args" : (d,)},
                      
                 {'type': 'eq',
                    'fun' : const3,
                      "args" : (d,) },
                
                 {'type': 'eq',
                     'fun' : const5,
                      "args" : (d,) },
                
                 {'type': 'eq',
                     'fun' : const6,
                      "args" : (d,) },
                
                 {'type': 'eq',
                     'fun' : const7,
                      "args" : (d,) }
                     ]
    
    res = sp.minimize(objective ,
                      x0, method='SLSQP',
               constraints = eq_constraints,
                      tol = 1e-8, args = (d,),
                bounds = [(0, 1) for i in range(3*d*d)])
    
    
    return res

In [11]:
result = main(2)

print(result)
print(-1*result.fun)
# x = [H,A2,B2]

     fun: -4.406151384719322e-09
     jac: array([-6.01039923e-09, -8.09550205e-11, -8.81230278e-09, -1.18694388e-10,
       -1.61093361e-13, -0.00000000e+00, -1.32772596e-04,  0.00000000e+00,
        0.00000000e+00, -8.81230278e-09,  0.00000000e+00, -1.11022302e-16])
 message: 'Singular matrix C in LSQ subproblem'
    nfev: 611
     nit: 46
    njev: 46
  status: 6
 success: False
       x: array([1.19743178e-09, 1.21135684e-09, 1.00000000e+00, 1.05032644e-08,
       4.52734529e-05, 1.00000000e+00, 6.63788474e-05, 3.50117776e-01,
       2.96473503e-04, 1.00000000e+00, 4.97657793e-01, 1.34691682e-02])
4.406151384719322e-09


In [ ]:
d_list = list(range(2,8))

Hardy  = []

for element in d_list:
    print(element)
    Hardy.append(main(element))
    
df = pd.DataFrame({"D": d_list, "Hardy": Hardy})
df = df.set_index("D")

print(df)

In [44]:
temp = []

for i in d_list:
    temp.append(-1*df["Hardy"][i].fun)

df["Optimal Value"] = temp

In [47]:
df["Entropy"] = -1*np.log2(df["Optimal Value"])

In [48]:
print(df)

                                               Hardy  Optimal Value    Entropy
D                                                                             
2  {'x': [5.019125443272991e-09, 8.43698667262983...   2.697640e-17  55.041080
3  {'x': [0.0, 5.888426848495118e-09, 2.053403024...   2.000000e+00  -1.000000
4  {'x': [2.4240118675204514e-08, 9.5864543372393...   4.000001e+00  -2.000000
5  {'x': [1.4245469488957776e-09, 7.9398003775805...   8.418835e+00  -3.073621
6  {'x': [1.3083256269997522e-12, 1.2495112719151...   1.309420e+01  -3.710856
7  {'x': [3.196924066567134e-07, 2.61148923479912...   1.967965e+01  -4.298633


In [ ]:
# A1 = [00,11] = {[1,000],[0,001]}
# B1 = [00,11]

# A2 = [ab,a'b'] #conditons
# B2 = [cd,c'd']

# h = [h1,h2,h3,h4]

# 3d^2 variables subject to some constraints

# h = d^2 vector [h is forced to be upper triangular]
# A2 = d vectors of size d
# B2 = d vectors of size d

# A1 = computational standard basis
# B2 = computational standard basis